# Optimal Control 16.323

In [109]:
import numpy as np
from functools import partial

# Lecture 3/4: Dynamic Programming and HJB

In [129]:
Nt = 10
Nx = [11,21]
n = len(Nx)
NX = Nx[0]*Nx[1]
Nu = 10
x1 = np.linspace(-2,2,Nx[0])
x2 = np.linspace(-1,4,Nx[1])
N = tuple([Nt] + Nx)
Nu = [10]
NU = Nu[0]
u = np.linspace(-2,2,NU)
Dt = .01
x_0 = np.array([[3],[-2]])
A = np.array([[1.5,0],[-.5,2]])
B = np.array([[0],[1]])
def xdot(xk,uk,k):
  return A@xk + uk
J = np.zeros(N) # J_k(x_k^i, x_k+1^j) k is time index
J[Nt-1,:,:] = np.outer(x1**2,2*x2**2)
print("J_T")
print(np.round(J[Nt-1,:,:],0))

J_T
[[  8.   4.   2.   0.   0.   0.   2.   4.   8.  12.  18.  24.  32.  40.
   50.  60.  72.  84.  98. 112. 128.]
 [  5.   3.   1.   0.   0.   0.   1.   3.   5.   8.  12.  16.  20.  26.
   32.  39.  46.  54.  63.  72.  82.]
 [  3.   2.   1.   0.   0.   0.   1.   2.   3.   4.   6.   9.  12.  15.
   18.  22.  26.  30.  35.  40.  46.]
 [  1.   1.   0.   0.   0.   0.   0.   1.   1.   2.   3.   4.   5.   6.
    8.  10.  12.  14.  16.  18.  20.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   1.   1.   2.
    2.   2.   3.   3.   4.   4.   5.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   1.   1.   1.   2.
    2.   2.   3.   3.   4.   5.   5.]
 [  1.   1.   0.   0.   0.   0.   0.   1.   1.   2.   3.   4.   5.   6.
    8.  10.  12.  14.  16.  18.  20.]
 [  3.   2.   1.   0.   0.   0.   1.   2.   3.   5.   6.   9.  12.  15.
   18.  22.  26.  30.  35.  41.  46.]
 [  5.

In [151]:

def gij_k(xi_k, uij_k, k):
  return 2*uij_k**2
def grid_interp(x_c,x_i,c_i):
  # x_i candidates Nxn where N candidates x in R^n
  # c_i is cost Nx1
  w_i = 1./np.linalg.norm(x_i-x_c,axis=1)
  print(w_i)
  print(np.dot(w_i,c_i))
  return np.dot(w_i,c_i)/np.sum(w_i)
  # demo pts
  # xc = np.array([2,3])
  # pts = np.array([[2.01,3.01],[1.66,2.53],[1.6,3.7]])
  # cpts = np.array([9.6,10.2,11.0])
  # print(grid_interp(xc,pts,cpts))


In [152]:
X_k1 = np.zeros((Nx + Nu + [2*n+1]))
# all possible states X all possible controls X [x^j_k+1 in R^n, interpolation shift, cost]
k = Nt-2
for i in range(NX): # all states
    l,m = divmod(i,Nx[1])
    xi = np.array([[x1[l]],[x2[m]]])
    xdot_uk = partial(xdot,xk=xi,k=k) # TODO jit
    for j in range(NU): # all controls
      uj_k = B*u[j]
      xdot_vec = xdot_uk(uk=uj_k)
      shift = -np.sign(xdot_vec*Dt)*np.ceil(np.abs(xdot_vec*Dt))
      X_k1[l,m,j,:n] = xdot_vec.flatten() #output
      X_k1[l,m,j,n:2*n] = shift.flatten()
      X_k1[l,m,j,-1] = gij_k(xi,u[j],k) # cost
    # output map is off-grid -> interpolate back
